In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from nltk.stem import PorterStemmer
import math
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
import string

In [ ]:
#loading the dataset 
df1 = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_interactions.csv')

In [ ]:
#This uses the loc method to select all rows from the DataFrame df1 where the 'review' column is not null. This results in a new DataFrame that contains only the rows with non-null values in the 'review' column.
df1 = df1.loc[~df1['review'].isnull()]

In [ ]:
#The output of this line will be a string that displays the number of unique users and unique recipes in the DataFrame.
n_users = len(df1.user_id.unique())
n_recipe = len(df1.recipe_id.unique())
print('Unique Users: {0}, unique recipe: {1}'.format(n_users, n_recipe))

In [ ]:
#The resulting user_counts Series can be used to analyze the distribution of ratings among users, and to identify users who have made the most or the fewest ratings.
user_counts = df1["user_id"].value_counts()
user_counts

In [ ]:
#The resulting active_users list can be used to filter the DataFrame df1 to include only the ratings or interactions made by the active users
active_users = user_counts.loc[user_counts >= 40].index.tolist()
active_users

negative and positive  words

In [ ]:
negative_type_words = ' '.join([text for text in df1['review'][ (df1['rating']==1) | (df1['rating']==2) ] ] )
positive_type_words = ' '.join([text for text in df1['review'][ (df1['rating']==4) | (df1['rating']==5) ] ] )

In [ ]:
punctuation_marks = set(string.punctuation)

In [ ]:
positive_type_words = ''.join([char for char in positive_type_words if char not in punctuation_marks])
negative_type_words = ''.join([char for char in negative_type_words if char not in punctuation_marks])

In [ ]:
import re
positive_type_words = re.sub(r'\d+', '', positive_type_words)
negative_type_words = re.sub(r'\d+', '', negative_type_words)

In [ ]:
negative_type_words = negative_type_words.replace("[^a-zA-Z]"," ")
positive_type_words = positive_type_words.replace("[^a-zA-Z]"," ")

In [ ]:
positive_type_words = positive_type_words.split()
negative_type_words = negative_type_words.split()

In [ ]:
positive_type_words = list(set(positive_type_words))
negative_type_words = list(set(negative_type_words))

In [ ]:
len(positive_type_words)

In [ ]:
len(negative_type_words)

In [ ]:
positive_type_words

In [ ]:
stopwords = stopwords.words('english')
positive_type_words = [word for word in positive_type_words if word not in stopwords]

In [ ]:
len(positive_type_words)

replacing 0 ratings with proper rating

In [ ]:
df = df1[(df1.user_id.isin(active_users))]
df

In [ ]:
df2 = df[df.rating==0]
df2

In [ ]:
def count_pos_words(text):
    pos_count = sum([text.lower().count(word) for word in positive_type_words])
    return pos_count

In [ ]:
df2['pos_count'] = df2['review'].apply(count_pos_words)

In [ ]:
def count_neg_words(text):
    neg_count = sum([text.lower().count(word) for word in negative_type_words])
    return neg_count

In [ ]:
df2['neg_count'] = df2['review'].apply(count_neg_words)

In [ ]:
df2

In [ ]:
for i in range(0,12585):
    if df2.iloc[i,5] == df2.iloc[i,6]:
        df2.iloc[i,3] = 3
    elif df2.iloc[i,5] > df2.iloc[i,6]:
        df2.iloc[i,3] = 4
    else :
        df2.iloc[i,3] = 2

In [ ]:
df2

In [ ]:
df2 = df2.drop(columns=['pos_count','neg_count'])

In [ ]:
df2

In [ ]:
df2.to_pickle('0-changed_rating.pkl')

In [ ]:
df = df[df.rating!=0]

In [ ]:
df

In [ ]:
frames = [df, df2]
result = pd.concat(frames)

In [ ]:
result

In [ ]:
result.to_pickle('final_data_to_work.pkl')

In [ ]:
result